In [1]:
import tensorflow as tf
tf.random.set_seed(10)
import tensorflow_datasets as tfds
dataset, dataset_info = tfds.load(name='malaria', shuffle_files=True, with_info=True, as_supervised=True, split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'])

2024-09-07 17:48:51.896245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-07 17:48:51.917246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-07 17:48:51.923650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-07 17:48:51.943605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1725731334.961564   38438 cuda_executor.c

In [2]:
# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])  # Resize images to 224x224
    image = tf.cast(image, tf.float32)  # Convert images to float32
    return image, label

# Apply preprocessing
train_dataset, valid_dataset, test_dataset = dataset

train_dataset = train_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# Normalization function
def normalise(image, label):
    return image / 255, label

# Apply normalization
train_dataset = train_dataset.map(normalise)
valid_dataset = valid_dataset.map(normalise)
test_dataset = test_dataset.map(normalise)

# Apply shuffling, batching, and prefetching
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [10]:
import wandb
import os

os.environ["WANDB_API_KEY"] = "wandbkey"
wandb.login()
wandb.init(project="Malaria-Detection", reinit=True)

wandb: Network error resolved after 0:08:15.953597, resuming normal operation.


In [21]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, InputLayer, RandomRotation, RandomFlip, Resizing, CenterCrop, BatchNormalization, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime


# Log the best hyperparameters to wandb
config = wandb.config
config.conv1_filters = 16
config.conv2_filters = 8
config.dense1_units = 256
config.dense2_units = 64
config.dropout1_rate = 0.3
config.dropout2_rate = 0.4
config.learning_rate = 0.0039852321056818825
import numpy as np
# TensorBoard log directory
log_dir = './logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Build the model with the best hyperparameters
model = Sequential()

# Input Layer
model.add(InputLayer(shape=(224, 224, 3)))

# Data Augmentation Layers
model.add(Resizing(224, 224))
model.add(RandomRotation(0.2))
model.add(RandomFlip('horizontal'))
model.add(CenterCrop(200, 200))

# Convolutional Layers with the best hyperparameters
model.add(Conv2D(filters=config.conv1_filters, kernel_size=3, padding='same', strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=config.conv2_filters, kernel_size=3, padding='same', strides=2, activation='relu'))
model.add(BatchNormalization())

# Global Pooling Layer
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(units=config.dense1_units, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=config.dropout1_rate))

model.add(Dense(units=config.dense2_units, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=config.dropout2_rate))

# Output Layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with the best learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [25]:
# Pass the Numpy arrays to WandbCallback
callbacks = [
    WandbMetricsLogger(),  # Logs metrics to wandb
    WandbModelCheckpoint(filepath="./models/best_model.keras", monitor="val_accuracy"),  # Saves best model
    tensorboard_callback,  # Logs to TensorBoard
    WandbCallback(validation_data=valid_dataset,labels=["Parasitized","Uninfected"])  # Disable graph logging
]


# Train the model
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=10,
    callbacks=callbacks
)

# Finish the wandb run
wandb.finish()

Epoch 1/10


AttributeError: 'Node' object has no attribute 'inbound_layers'